#### Exploratory Data Analysis

Dataset: 

- _vehicles_us.csv_

Author: Luis Sergio Pastrana Lemus  
Date: 2025-06-24

# Exploratory Data Analysis – Purchasing Activity Dataset

## __1. Libraries__

In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## __2. Path to Data file__

In [ ]:
# Build route to data file and upload
df_vehicles = pd.read_csv(
    "../Datasets/vehicles_us.csv", sep=',', header='infer')

## __3. Exploratory Data Analysis__

### 3.1  Descriptive Statistics

In [3]:
df_vehicles

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013.0,nissan maxima,like new,6.0,gas,88136.0,automatic,sedan,black,NaN,2018-10-03,37
51521,2700,2002.0,honda civic,salvage,4.0,gas,181500.0,automatic,sedan,white,NaN,2018-11-14,22
51522,3950,2009.0,hyundai sonata,excellent,4.0,gas,128000.0,automatic,sedan,blue,NaN,2018-11-15,32
51523,7455,2013.0,toyota corolla,good,4.0,gas,139573.0,automatic,sedan,black,NaN,2018-07-02,71


#### 3.1.1 Descriptive statistics for Distribution of prices and mileage

In [ ]:
# Selection widget: view mode
view_selector = widgets.RadioButtons(
    options=['Price only', 'Odometer only', 'Both'], value='Both', description='View:')


# Selection widget: view mode
view_selector = widgets.RadioButtons(
    options=['Price only', 'Odometer only', 'Both'], value='Both', description='View:')

# Update function


def update_graph(view_mode):

    if view_mode == 'Price only':

        fig = px.histogram(df_vehicles, x='price', nbins=50, title='Price Distribution',
                           color_discrete_sequence=['grey'])

    elif view_mode == 'Odometer only':

        fig = px.histogram(df_vehicles, x='odometer', nbins=50, title='Odometer Distribution',
                           color_discrete_sequence=['grey'])

    else:

        # Combined in subplots
        fig_price = px.histogram(df_vehicles, x='price', nbins=50,
                                 color_discrete_sequence=['grey'])
        fig_odometer = px.histogram(df_vehicles, x='odometer', nbins=50,
                                    color_discrete_sequence=['grey'])

        fig = make_subplots(
            rows=1, cols=2, subplot_titles=("Price", "Odometer"))
        fig.add_trace(fig_price.data[0], row=1, col=1)
        fig.add_trace(fig_odometer.data[0], row=1, col=2)
        fig.update_layout(
            title_text="Distribution of Price and Odometer", showlegend=False)

    fig.show()


# Run interactive display
widgets.interact(update_graph, view_mode=view_selector)

interactive(children=(RadioButtons(description='View:', index=2, options=('Price only', 'Odometer only', 'Both…

<function __main__.update_graph(view_mode)>

#### 3.1.2 Descriptive statistics for Comparison by vehicle type or condition

In [ ]:
# Dropdown widget: compare by 'type' or 'condition'
compare_selector = widgets.Dropdown(options=[('Vehicle Type', 'type'), (
    'Condition', 'condition')], value='type', description='Compare by:')

# Update function for bar chart


def update_bar_chart(group_by_col):

    # Group and count
    grouped = df_vehicles[group_by_col].value_counts().reset_index()
    grouped.columns = [group_by_col, 'count']

    # Create bar chart
    fig = px.bar(grouped, x=group_by_col, y='count', title=f'Number of Listings by {group_by_col.capitalize()}',
                 labels={group_by_col: group_by_col.capitalize(), 'count': 'Listings'}, color_discrete_sequence=['grey'])
    fig.update_layout(xaxis={'categoryorder': 'total descending'})
    fig.show()


# Interactive widget display
widgets.interact(update_bar_chart, group_by_col=compare_selector)

interactive(children=(Dropdown(description='Compare by:', options=(('Vehicle Type', 'type'), ('Condition', 'co…

<function __main__.update_bar_chart(group_by_col)>

#### 3.1.3 Descriptive statistics for Scatter plots for price vs. model year or mileage

In [ ]:
# Dropdown for choosing x-axis
xaxis_selector = widgets.Dropdown(options=[('Model Year', 'model_year'), ('Mileage (Odometer)', 'odometer')], value='model_year',
                                  description='X-axis:')

# Update function


def update_scatter(x_axis):

    fig = px.scatter(df_vehicles, x=x_axis, y='price', title=f'Price vs. {x_axis.replace("_", " ").title()}',
                     labels={x_axis: x_axis.replace("_", " ").title(), 'price': 'Price'}, opacity=0.6,
                     hover_data=['model', 'condition', 'fuel'], color_discrete_sequence=['grey'])

    fig.show()


# Display the interactive widget
widgets.interact(update_scatter, x_axis=xaxis_selector)

interactive(children=(Dropdown(description='X-axis:', options=(('Model Year', 'model_year'), ('Mileage (Odomet…

<function __main__.update_scatter(x_axis)>

#### 3.1.4 Descriptive statistics for Price Trends Over Time

In [ ]:
# Dropdown widget for aggregation method
agg_selector = widgets.Dropdown(options=[(
    'Mean', 'mean'), ('Median', 'median')], value='mean', description='Aggregate:')

# Update function


def update_price_trend(agg_func):

    df_filtered = df_vehicles.dropna(subset=['model_year', 'price'])

    if agg_func == 'mean':

        grouped = df_filtered.groupby('model_year')[
            'price'].mean().reset_index()

    else:

        grouped = df_filtered.groupby('model_year')[
            'price'].median().reset_index()

    fig = px.line(grouped, x='model_year', y='price', title=f'Average Price by Model Year ({agg_func.title()})', markers=True,
                  labels={'model_year': 'Model Year', 'price': 'Price'}, color_discrete_sequence=['grey'])
    fig.show()


# Display the interactive widget
widgets.interact(update_price_trend, agg_func=agg_selector)

interactive(children=(Dropdown(description='Aggregate:', options=(('Mean', 'mean'), ('Median', 'median')), val…

<function __main__.update_price_trend(agg_func)>

#### 3.1.5 Descriptive statistics for Condition Impact

In [ ]:
# Dropdown: Group by model or brand (if 'brand' is a separate column; otherwise use 'model')
group_selector = widgets.Dropdown(
    options=[('Model', 'model')], value='model', description='Group by:')

# Slider for top N entries
top_n_slider = widgets.IntSlider(
    value=10, min=5, max=30, step=1, description='Top N Models:')

# Update function


def update_condition_impact(group_col, top_n):

    df_filtered = df_vehicles.dropna(subset=[group_col, 'condition', 'price'])

    # Compute average price
    grouped = df_filtered.groupby([group_col, 'condition'])[
        'price'].mean().reset_index()

    # Filter top N by total listing count to avoid clutter
    top_models = df_filtered[group_col].value_counts().nlargest(top_n).index
    grouped = grouped[grouped[group_col].isin(top_models)]

    fig = px.bar(grouped, x=group_col, y='price', color='condition', barmode='group',
                 title=f'Average Price by {group_col.capitalize()} and Condition',
                 labels={group_col: group_col.capitalize(
                 ), 'price': 'Average Price', 'condition': 'Condition'},
                 color_discrete_sequence=['#e0e0e0', '#d9d9d9', '#bfbfbf', '#a6a6a6', '#7f7f7f', "#555555"])
    fig.show()


# Interactive widget
widgets.interact(update_condition_impact,
                 group_col=group_selector, top_n=top_n_slider)

interactive(children=(Dropdown(description='Group by:', options=(('Model', 'model'),), value='model'), IntSlid…

<function __main__.update_condition_impact(group_col, top_n)>

#### 3.1.6 Descriptive statistics for Brand vs. Price

In [23]:
# Extract brand from model (if brand column doesn't exist)
df_vehicles['brand'] = df_vehicles['model'].str.split().str[0]

In [ ]:
# Slider for top N brands
top_n_slider = widgets.IntSlider(
    value=10, min=5, max=30, step=1, description='Top N Brands:')

# Update function


def update_brand_price_chart(top_n):

    df_clean = df_vehicles.dropna(subset=['brand', 'price'])

    # Get top N brands by count
    top_brands = df_clean['brand'].value_counts().nlargest(top_n).index
    filtered = df_clean[df_clean['brand'].isin(top_brands)]

    # Compute average price
    avg_price = filtered.groupby('brand')['price'].mean().reset_index()

    fig = px.bar(avg_price, x='brand', y='price', title=f'Average Price by Brand (Top {top_n})',
                 labels={'brand': 'Brand', 'price': 'Average Price'}, color_discrete_sequence=['#808080'])
    fig.update_layout(xaxis={'categoryorder': 'total descending'})
    fig.show()


# Display interactive widget
widgets.interact(update_brand_price_chart, top_n=top_n_slider)

interactive(children=(IntSlider(value=10, description='Top N Brands:', max=30, min=5), Output()), _dom_classes…

<function __main__.update_brand_price_chart(top_n)>

#### 3.1.7 Descriptive statistics for Mileage vs. Price (Depreciation Curve)

In [ ]:
# Dropdown for filter category
filter_selector = widgets.Dropdown(options=['All'] + df_vehicles['condition'].dropna().unique().tolist(), value='All',
                                   description='Condition:')

# Function to update plot based on condition


def update_heatmap(selected_condition):

    # Filter
    if selected_condition == 'All':

        df_filtered = df_vehicles[['odometer', 'price']].dropna()

    else:

        df_filtered = df_vehicles[df_vehicles['condition'] ==
                                  selected_condition][['odometer', 'price']].dropna()

    # Remove top 1% outliers
    df_filtered = df_filtered[df_filtered['price']
                              < df_filtered['price'].quantile(0.99)]
    df_filtered = df_filtered[df_filtered['odometer']
                              < df_filtered['odometer'].quantile(0.99)]

    # Plot
    fig = px.density_heatmap(df_filtered, x='odometer', y='price',
                             nbinsx=50, nbinsy=50,
                             title=f'Price vs. Mileage Heatmap (Condition: {selected_condition})',
                             labels={
                                 'odometer': 'Mileage (Odometer)', 'price': 'Price'},
                             color_continuous_scale='Greys')
    fig.show()


# Display interactive widget
widgets.interact(update_heatmap, selected_condition=filter_selector)

interactive(children=(Dropdown(description='Condition:', options=('All', 'good', 'like new', 'fair', 'excellen…

<function __main__.update_heatmap(selected_condition)>

#### 3.1.8 Descriptive statistics for Top Models Sold

In [ ]:
# Dropdown: group by model or brand
group_selector = widgets.Dropdown(options=[(
    'Model', 'model'), ('Brand', 'brand')], value='model', description='Group by:')

# Slider for top N entries
top_n_slider = widgets.IntSlider(
    value=10, min=5, max=30, step=1, description='Top N:')

# Update function


def update_pie_chart(group_by, top_n):

    df_filtered = df_vehicles.dropna(subset=[group_by])
    top_counts = df_filtered[group_by].value_counts().nlargest(
        top_n).reset_index()
    top_counts.columns = [group_by, 'count']

    # Sort ascending to match lighter to lower values
    top_counts = top_counts.sort_values('count')

    fig = px.pie(top_counts, names=group_by, values='count',
                 title=f'Share of Top {top_n} {group_by.capitalize()}s Listed',
                 color_discrete_sequence=px.colors.sequential.Greys[3:])

    fig.show()


# Display interactive widget
widgets.interact(update_pie_chart, group_by=group_selector, top_n=top_n_slider)

interactive(children=(Dropdown(description='Group by:', options=(('Model', 'model'), ('Brand', 'brand')), valu…

<function __main__.update_pie_chart(group_by, top_n)>

#### 3.1.9 Descriptive statistics for Transmission Type Analysis

In [ ]:
# Dropdown to select aggregation method
agg_selector = widgets.Dropdown(options=[(
    'Mean', 'mean'), ('Median', 'median')], value='mean', description='Aggregate:')

# Update function


def update_price_by_trans(agg_func):

    df_trans = df_vehicles.dropna(subset=['transmission', 'price'])

    # Group by and calculate aggregation
    if agg_func == 'mean':

        grouped = df_trans.groupby('transmission')[
            'price'].mean().reset_index()

    else:

        grouped = df_trans.groupby('transmission')[
            'price'].median().reset_index()

    grouped = grouped.sort_values('price', ascending=True)

    # Horizontal bar chart
    fig = px.bar(grouped, y='transmission', x='price',
                 orientation='h',
                 title=f'{agg_func.title()} Price by Transmission Type',
                 labels={'transmission': 'Transmission',
                         'price': f'{agg_func.title()} Price'},
                 color_discrete_sequence=['#808080'])

    fig.update_layout(yaxis=dict(categoryorder='total ascending'))
    fig.show()


# Display widget
widgets.interact(update_price_by_trans, agg_func=agg_selector)

interactive(children=(Dropdown(description='Aggregate:', options=(('Mean', 'mean'), ('Median', 'median')), val…

<function __main__.update_price_by_trans(agg_func)>

#### 3.1.10 Descriptive statistics for Fuel Type Preference

In [ ]:
# Dropdown for selecting aggregation focus (optionally)
fuel_selector = widgets.SelectMultiple(options=df_vehicles['fuel'].dropna().unique().tolist(), value=['gas', 'diesel', 'electric', 'hybrid'],
                                       description='Fuel Types:', layout=widgets.Layout(width='50%'), style={'description_width': 'initial'})

# Update function


def update_fuel_price_distribution(selected_fuels):

    df_filtered = df_vehicles[df_vehicles['fuel'].isin(
        selected_fuels)].dropna(subset=['price', 'fuel'])

    # Remove top outliers for better scale (optional)
    df_filtered = df_filtered[df_filtered['price']
                              < df_filtered['price'].quantile(0.99)]

    fig = px.box(df_filtered, x='fuel', y='price',
                 title='Price Distribution by Fuel Type',
                 labels={'fuel': 'Fuel Type', 'price': 'Price'},
                 color='fuel', color_discrete_sequence=px.colors.sequential.Blackbody)

    fig.update_layout(xaxis_title='Fuel Type', yaxis_title='Price')
    fig.show()


# Display interactive widget
widgets.interact(update_fuel_price_distribution, selected_fuels=fuel_selector)

interactive(children=(SelectMultiple(description='Fuel Types:', index=(0, 1, 4, 3), layout=Layout(width='50%')…

<function __main__.update_fuel_price_distribution(selected_fuels)>

#### 3.1.11 Descriptive statistics for Correlation Analysis of Vehicle Features

In [ ]:
# Prepare and encode
df_corr = df_vehicles.copy()

# Encode categorical features
condition_map = {'salvage': 1, 'fair': 2, 'good': 3,
                 'excellent': 4, 'like new': 5, 'new': 6}
fuel_map = {'gas': 1, 'diesel': 2, 'hybrid': 3, 'electric': 4, 'other': 0}
transmission_map = {'manual': 0, 'automatic': 1, 'other': 0.5}

df_corr['condition_score'] = df_corr['condition'].map(condition_map)
df_corr['fuel_score'] = df_corr['fuel'].map(fuel_map)
df_corr['transmission_score'] = df_corr['transmission'].map(transmission_map)

# Variable selector
all_vars = {
    'Price': 'price',
    'Odometer': 'odometer',
    'Model Year': 'model_year',
    'Days Listed': 'days_listed',
    'Is 4WD': 'is_4wd',
    'Condition Score': 'condition_score',
    'Fuel Score': 'fuel_score',
    'Transmission Score': 'transmission_score'
}

var_selector = widgets.SelectMultiple(
    options=all_vars.keys(),
    value=('Price', 'Odometer', 'Model Year', 'Condition Score'),
    description='Variables:',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

# Update function


def update_scatter_matrix(selected_labels):
    selected_cols = [all_vars[label] for label in selected_labels]
    df_selected = df_corr[selected_cols].dropna()

    fig = px.scatter_matrix(df_selected,
                            dimensions=selected_cols,
                            title="Scatter Matrix of Selected Features",
                            color_discrete_sequence=['grey'])

    fig.update_traces(diagonal_visible=True, showupperhalf=False)
    fig.show()


# Display widget
widgets.interact(update_scatter_matrix, selected_labels=var_selector)

interactive(children=(SelectMultiple(description='Variables:', index=(0, 1, 2, 5), layout=Layout(width='50%'),…

<function __main__.update_scatter_matrix(selected_labels)>